In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [13]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/rtc-directory')
time.sleep(3)
driver.maximize_window()
redbus_dic= driver.find_element(By.CSS_SELECTOR,'a[href="/online-booking/ktcl"]')
redbus_dic .click()
KTCL_route_names=[]
KTCL_route_links=[] 
def scrape_data(): 
    route_names= driver.find_elements (By.CLASS_NAME, "route") 
    for route_name in route_names: 
        KTCL_route_names.append(route_name.text)
    route_links= driver.find_elements (By.CLASS_NAME, "route")
    hrefs =[route_link.get_attribute('href') for route_link in route_links] 
    for href in hrefs:
        KTCL_route_links.append(href)
page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")

print(len(page_tabs))



for i in range(1, len(page_tabs)+1):
    if i > 0:
        x_path = f'/html/body/div[1]/div/div[4]/div[12]/div[{i}]' 
        element=driver.find_element(By.XPATH, x_path)
        driver.execute_script("arguments[0].click();", element)
        scrape_data()
        
print(KTCL_route_names)
print(KTCL_route_links)



4
['Pune to Goa', 'Goa to Pune', 'Mumbai to Goa', 'Goa to Mumbai', 'Pandharpur to Goa', 'Bangalore to Goa', 'Goa to Pandharpur', 'Belagavi to Goa', 'Goa to Bangalore', 'Solapur to Goa', 'Goa to Kolhapur(Maharashtra)', 'Goa to Solapur', 'Goa to Sangola (Solapur)', 'Sangola (Solapur) to Goa', 'Calangute (goa) to Goa Airport', 'Goa to Sangli', 'Calangute (goa) to Mopa Airport', 'Mopa Airport to Calangute (goa)', 'Ponda to Belagavi', 'Goa to Miraj', 'Goa Airport to Calangute (goa)', 'Marcel to Belagavi', 'Shivamogga to Goa', 'Goa to Mopa Airport', 'Goa to Satara', 'Belagavi to Marcel', 'Mopa Airport to Goa', 'Shirdi to Goa', 'Goa to Shivamogga', 'Goa to Shirdi', 'Goa to Goa Airport', 'Margao to Mopa Airport', 'Goa Airport to Goa', 'Mopa Airport to Margao', 'Belagavi to Saquelim', 'Panaji to Mopa Airport', 'Saquelim to Belagavi', 'Calangute (goa) to Goa', 'Calangute (goa) to Panaji', 'Goa Airport to Panaji']
['https://www.redbus.in/bus-tickets/pune-to-goa', 'https://www.redbus.in/bus-ticket

In [14]:
bus_names = []
bus_types = []
bus_rating = []
dep_time = []
avl_time = []
duration = []
available_seat = []
price = []
route_names = []
route_links = []

def scroll_down():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

for route_name, route_link in zip(KTCL_route_names, KTCL_route_links):
    driver.get(route_link)
    time.sleep(1)
    try:
        view_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'button')))
        view_more_button.click()
    except Exception as e:
        print(f"View more button not found for {route_link}: {e}")
        continue
   
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        scroll_down()
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    bus_name_elements = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
    bus_type_elements = driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')
    rating_elements = driver.find_elements(By.XPATH, '//div[@class="rating-sec lh-24"]')
    depature_elements = driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')
    arrival_elements = driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')
    duration_elements = driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')
    seat_elements = driver.find_elements(By.XPATH, '//div[@class="column-eight w-15 fl"]')
    rate_elements = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
    
    
    for bus in bus_name_elements:
        bus_names.append(bus.text)
        route_names.append(route_name)
        route_links.append(route_link)
    for bus_type in bus_type_elements:
        bus_types.append(bus_type.text)
    for rating in rating_elements:
        bus_rating.append(rating.text)
    for dep in depature_elements:
        dep_time.append(dep.text)
    for arr in arrival_elements:
        avl_time.append(arr.text)
    for dur in duration_elements:
        duration.append(dur.text)
    for seat in seat_elements:
        available_seat.append(seat.text)
    for fare in rate_elements:
        price.append(fare.text)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        scroll_down()
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


max_length = max(len(bus_names), len(bus_types), len(bus_rating), len(dep_time), len(avl_time), len(duration), len(available_seat), len(price))

bus_names.extend([None] * (max_length - len(bus_names)))
bus_types.extend([None] * (max_length - len(bus_types)))
bus_rating.extend([None] * (max_length - len(bus_rating)))
dep_time.extend([None] * (max_length - len(dep_time)))
avl_time.extend([None] * (max_length - len(avl_time)))
duration.extend([None] * (max_length - len(duration)))
available_seat.extend([None] * (max_length - len(available_seat)))
price.extend([None] * (max_length - len(price)))


data = {
    "Route Name": route_names,
    "Route Link": route_links,
    'Bus Name': bus_names,
    'Bus Type': bus_types,
    'Bus Rating': bus_rating,
    'Departure Time': dep_time,
    'Arrival Time': avl_time,
    'Duration': duration,
    'Available Seats': available_seat,
    'Price': price
}

df = pd.DataFrame(data)
df.to_csv('kadamba_bus_details.csv', index=False)
print(df.head())


    Route Name                                     Route Link  \
0  Pune to Goa  https://www.redbus.in/bus-tickets/pune-to-goa   
1  Pune to Goa  https://www.redbus.in/bus-tickets/pune-to-goa   
2  Pune to Goa  https://www.redbus.in/bus-tickets/pune-to-goa   
3  Pune to Goa  https://www.redbus.in/bus-tickets/pune-to-goa   
4  Pune to Goa  https://www.redbus.in/bus-tickets/pune-to-goa   

                                            Bus Name  \
0  Kadamba Transport Corporation Limited (KTCL) -...   
1                                      Atmaram Gobus   
2                                     Ashray Travels   
3                                      Atmaram Gobus   
4                                    Ashray Amrutyog   

                        Bus Type Bus Rating Departure Time Arrival Time  \
0              A/C Sleeper (2+1)        3.7          19:15        05:50   
1           VE A/C Sleeper (2+1)        4.6          22:30        08:45   
2  Bharat Benz A/C Sleeper (2+1)        4.6    